In [216]:
!pip install clickhouse-driver

In [217]:
#Импорт библиотек
import requests as request
import pandas as pd
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from clickhouse_driver import Client 
import json

In [218]:
#URL и дата для извлечения данных
URL = 'https://api.exchangerate.host/timeframe'
parameters = {'access_key': '043dc9dad696914726d3064e9d917294',
              'source': 'USD', 
              'start_date': '2023-01-01',
              'end_date': '2023-01-01'

}

In [219]:
#Подключение к базе 
# Настройка подключения к базе данных ClickHouse
CH_CLIENT = Client(
    host='158.160.116.58',  # IP-адрес сервера ClickHouse
    user='student',  # Имя пользователя для подключения
    password='dfqh89fhq8',  # Пароль для подключения
    database='sandbox'  # База данных, к которой подключаемся
)

In [220]:
#Извлекаем данные
def extract_data(url, file):
    response = request.get(URL, params=parameters)
    data = response.json()
    with open(file, 'w', encoding='utf-8') as f:
        json.dump(data, f)
    
#Обрабатываем данные и записываем их в csv файл

def transform_data(file, csv_file):
    with open(file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    rows = []
    source = data['source']
    for date, pairs in data['quotes'].items():
        for pair, rate in pairs.items():
            currency = pair[3:]
            rows.append({'date': date, 
                         'currency_source': source, 
                         'currency': currency, 
                         'value': rate})


    df = pd.DataFrame(rows, columns=['date', 'currency_source', 'currency', 'value'])
    df.to_csv(csv_file, sep=';', encoding='utf-8', index=False)

In [221]:
#Загружаем данные в CH

def upload_to_clickhouse(csv_file, table_name, client):
    df = pd.read_csv(csv_file, delimiter=';')
    client.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (date String, currency_source String, currency String, value Float64) ENGINE = Log")
    client.execute(f"INSERT INTO {table_name} VALUES", df.to_dict('records'))

In [222]:
#Проверка правильности загрузки данных

def check_if_correct(query, client):
    return client.execute(query)

In [223]:
#Выполняем функции

extract_data(URL, 'currency_file')

transform_data("currency_file", "currency_exchange.csv")

upload_to_clickhouse('currency_exchange.csv','currency_exchange666', CH_CLIENT)

check_if_correct("SELECT * FROM currency_exchange666 WHERE currency = 'GYD'", CH_CLIENT)

[('2023-01-01', 'USD', 'GYD', 209.232614),
 ('2023-01-01', 'USD', 'GYD', 209.232614),
 ('2023-01-01', 'USD', 'GYD', 209.232614)]